In [1]:
import yfinance as yf 
import pandas as pd 
from datetime import datetime, timedelta 
import numpy as np 
from scipy.optimize import minimize

tickers = ['BND','AMZN','MSFT','NVDA','PFE','NEE','DIS','MCD','JPM','BAC','CAC.PA','GBPUSD=X']

end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)

adj_close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
    adj_close_df[ticker] = data['Close']

print(adj_close_df)
    
log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()

cov_matrix = log_returns.cov()*252
print(cov_matrix)

def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

def expected_return(weights, log_returns):
                     return np.sum(log_returns.mean()*weights)*252

def sharp_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)

risk_free_rate = 0.02

def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharp_ratio(weights, log_returns, cov_matrix, risk_free_rate)

min_weight_bnd = 0.1
min_weight_gbpusd = 0.05

constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
    {'type': 'ineq', 'fun': lambda x: x[tickers.index('BND')] - min_weight_bnd},
    {'type': 'ineq', 'fun': lambda x: x[tickers.index('GBPUSD=X')] - min_weight_gbpusd}
]

bounds = [(0, 0.5) for _ in range(len(tickers))]

initial_weights = np.array([1/len(tickers)] * len(tickers))
print(initial_weights)

optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)
   

optimal_weights = optimized_results.x

print("optimal weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

print()
optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharp_ratio = sharp_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Returns: {optimal_portfolio_return:.4f}")
print(f"Expected volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharp Ratio: {optimal_sharp_ratio:.4f}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                  BND        AMZN        MSFT        NVDA        PFE  \
Date                                                                   
2020-05-15  75.800987  120.488998  175.316849    8.459581  28.484474   
2020-05-18  75.662117  121.313004  176.991928    8.718129  28.718330   
2020-05-19  75.757599  122.466499  175.766754    8.773177  28.424128   
2020-05-20  75.957169  124.897003  178.204742    8.937071  28.386414   
2020-05-21  76.035271  122.336998  176.064270    8.743039  28.107296   
...               ...         ...         ...         ...        ...   
2025-05-07  72.900002  188.710007  433.350006  117.059998  22.363371   
2025-05-08  72.510002  192.080002  438.170013  117.370003  22.539999   
2025-05-09  72.570000  193.059998  438.730011  116.650002  22.280001   
2025-05-12  72.320000  208.639999  449.260010  123.000000  23.090000   
2025-05-13  72.290001  211.369995  449.140015  129.929993  22.860100   

                  NEE         DIS         MCD         JPM      

In [36]:

import numpy as np
import pandas as pd
import yfinance as yf 
from pandas_datareader import data as web 
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt 
import mplfinance as mpf


def save_to_csv_from_yahoo(ticker, syear, smonth, sday, eyear, emonth, eday):
    start = dt.datetime(syear, smonth, sday)
    end = dt.datetime(eyear, emonth, eday)
    df = yf.download(ticker,start=start, end=end)
    df.to_csv(r'C:\Users\asus\OneDrive\Bureau\MesProjetsFinance//' + ticker + '.csv')
    return df

save_to_csv_from_yahoo('QQQ' ,2010, 1, 1, 2020, 1, 1)

def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(r'C:\Users\asus\OneDrive\Bureau\MesProjetsFinance//' + ticker + '.csv')
    except FileNotFoundError:
        print("file doesn't exist")
    else:
        return df
    
get_df_from_csv('QQQ')








[*********************100%***********************]  1 of 1 completed


,Price,Close,High,Low,Open,Volume
0,Ticker,QQQ,QQQ,QQQ,QQQ,QQQ
1,Date,NaN,NaN,NaN,NaN,NaN
2,2010-01-04,40.485801696777344,40.54685616287728,40.35497925323117,40.40731022687217,62822800
3,2010-01-05,40.485801696777344,40.55557632829868,40.25904079840811,40.459637873475614,62935600
4,2010-01-06,40.24161911010742,40.59920597428198,40.18056793771465,40.46838345931433,96033000
...,...,...,...,...,...,...
2513,2019-12-24,205.21995544433594,205.38457896716534,204.7551352165606,205.29742794502658,7089000
2514,2019-12-26,207.03082275390625,207.05019457258925,205.5201459767313,205.5491963165717,17067500
2515,2019-12-27,206.85650634765625,207.77646819023298,206.30452037628993,207.75709637289543,18134100
2516,2019-12-30,205.50079345703125,206.87589708642446,204.48398721113412,206.75000242676649,21815500


In [12]:
import numpy as np 
import pandas as pd 
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt 
from scipy.stats import norm 

years = 15 

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days = 365*years)

tickers = ['SPY', 'BND', 'GLD','QQQ', 'VTI']

adj_close_df = pd.DataFrame()
for ticker in tickers:
    data = yf.download(ticker, start=startDate, end=endDate, auto_adjust=True)
    adj_close_df[ticker] = data['Close']


print(adj_close_df)


log_returns = np.log(adj_close_df/adj_close_df.shift(1))
log_returns = log_returns.dropna()
print(log_returns)

portfolio_value = 1000000
weights = np.array([1/len(tickers)]*len(tickers))
print(weights)

historical_returns = (log_returns * weights).sum(axis=1)
print(historical_returns)


days = 5 
range_returns = historical_returns.rolling(window = days).sum()
range_returns = range_returns.dropna()
print(range_returns)

confidence_interval = 0.95

VaR = -np.percentile(range_returns, 100 - (confidence_interval * 100))*portfolio_value
print(VaR)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                   SPY        BND         GLD         QQQ         VTI
Date                                                                 
2010-05-25   81.934494  52.538460  117.360001   39.027306   42.010784
2010-05-26   81.440552  52.473389  118.470001   38.590752   41.835125
2010-05-27   84.168648  52.271610  118.690002   40.048832   43.301418
2010-05-28   83.112373  52.375744  118.879997   39.813095   42.835560
2010-06-01   81.714127  52.417477  119.910004   39.446388   41.949684
...                ...        ...         ...         ...         ...
2025-05-14  587.590027  72.040001  293.160004  518.679993  289.410004
2025-05-15  590.460022  72.430000  297.839996  519.250000  290.589996
2025-05-16  594.200012  72.480003  294.239990  521.510010  292.579987
2025-05-19  594.849976  72.459999  298.029999  522.010010  292.809998
2025-05-20  592.849976  72.330002  303.579987  520.270020  291.850006

[3771 rows x 5 columns]
                 SPY       BND       GLD       QQQ       VTI
Date